# RAG Pipeline with LangChain

## 1. Setup and Imports

In [1]:
import os
import json
import ast
from dotenv import load_dotenv
from konlpy.tag import Okt

# LangChain components
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain_classic.retrievers import EnsembleRetriever
from langchain_text_splitters import RecursiveCharacterTextSplitter

### API Key Configuration

In [2]:
load_dotenv()
try:
    api_key = os.getenv("GOOGLE_API_KEY")
    if not api_key: 
        raise ValueError("GOOGLE_API_KEY not set in .env file.")
    print("API key loaded successfully.")
except ValueError as e:
    print(e)

API key loaded successfully.


## 2. Data Loading and Document Preparation
Load the audit cases and prepare them in LangChain's `Document` format. We create two sets of documents for our hybrid search strategy:
1.  **Full-text documents:** For semantic search (FAISS).
2.  **Keyword-focused documents:** For keyword search (BM25), using only the 'problem' and 'action' fields.

In [ ]:
def load_and_prepare_docs(filepath="audit_cases.json"):
    """
    'contents_summary' 필드만을 사용하여 문서를 생성
    """
    print(f"Loading data from {filepath} and creating baseline documents from summaries...")
    with open(filepath, 'r', encoding='utf-8') as f:
        audit_cases = json.load(f)

    docs = []

    for i, case in enumerate(audit_cases):
        # 1. 메타데이터 추출
        site = case.get('site', '알 수 없음')
        category = case.get('category', '알 수 없음')
        date = case.get('date', '알 수 없음')
        original_title = case.get('title', '')

        metadata = {
            "index": i, "title": original_title, "site": site,
            "category": category, "date": date
        }

        # 2. 'contents_summary'를 기반으로 문서 내용 생성
        summary_dict = {}
        summary_str = case.get('contents_summary')
        if summary_str:
            try:
                summary_dict = ast.literal_eval(summary_str)
            except (ValueError, SyntaxError):
                summary_dict = {}

        # 요약 필드들을 구조화된 텍스트로 조합
        title = summary_dict.get('title_str', original_title)
        keywords = ", ".join(summary_dict.get('keyword_list', []))
        problems = summary_dict.get('problems_str', '')
        action = summary_dict.get('action_str', '')
        standards = summary_dict.get('standards_str', '')

        summary_based_text = (
            f"출처: {site}\n"
            f"분류: {category}\n"
            f"일자: {date}\n"
            f"제목: {title}\n"
            f"핵심 키워드: {keywords}\n"
            f"문제 요약: {problems}\n"
            f"조치 요약: {action}\n"
            f"관련 규정: {standards}"
        )

        docs.append(Document(page_content=summary_based_text, metadata=metadata))

    # 단순한 기준점을 위해, 두 리트리버에 동일한 문서를 제공
    full_text_documents = docs
    keyword_documents = docs

    print(f"  - Created {len(docs)} summary-based documents for baseline.")
    return full_text_documents, keyword_documents

full_text_documents, keyword_documents = load_and_prepare_docs()

Loading data from audit_cases.json and creating baseline documents from summaries...
  - Created 4961 summary-based documents for baseline.


## 3. Retriever Setup (Hybrid Search)
We'll set up two retrievers and combine them using `EnsembleRetriever`.

In [4]:
# Initialize models and tokenizers
print("Initializing models and retrievers...")
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
okt = Okt()

# 1. FAISS (Semantic) Retriever
print("  - Building FAISS index...")
faiss_vectorstore = FAISS.from_documents(full_text_documents, embeddings)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 5})
print("    - FAISS retriever ready.")

# 2. BM25 (Keyword) Retriever
print("  - Building BM25 index...")
bm25_retriever = BM25Retriever.from_documents(
    documents=keyword_documents, 
    preprocess_func=lambda s: okt.morphs(s) # Use Okt for tokenization
)
bm25_retriever.k = 5
print("    - BM25 retriever ready.")

# 3. Ensemble (Hybrid) Retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[faiss_retriever, bm25_retriever],
    weights=[0.5, 0.5] # Give equal weight to semantic and keyword search
)
print("Ensemble retriever ready!")

Initializing models and retrievers...
  - Building FAISS index...
    - FAISS retriever ready.
  - Building BM25 index...
    - BM25 retriever ready.
Ensemble retriever ready!


## 4. RAG Chain Construction (LCEL)
Now we define the full RAG chain using LangChain Expression Language (LCEL).

In [5]:
prompt_template = """
당신은 감사 전문가입니다. 제공되는 '관련 감사 사례'를 근거로 하여 사용자의 '질문'에 대해 답변해 주세요. 근거가 부족하면 '정보 없음'으로 답하세요.

[관련 감사 사례]
{context}

[질문]
{question}

[답변]
"""

prompt = ChatPromptTemplate.from_template(prompt_template)
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

# Helper function to format retrieved documents
def format_docs(docs):
    return "\n\n".join([f"### 감사사례 (제목: {doc.metadata.get('title', 'N/A')}){doc.page_content}" for doc in docs])

# RAG Chain
rag_chain = (
    {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("RAG chain constructed successfully.")

RAG chain constructed successfully.


## 5. Test Queries
**이 셀의 `test_query` 변수만 변경하고 이 셀만 반복적으로 실행하여 다양한 질문을 테스트할 수 있습니다.**

In [6]:
test_query = "부실시공에 따라 재시공을 하도록 한 감사건도 있나?" # 여기에 질문을 변경하세요!

print(f"--- Running RAG chain for query: '{test_query}' ---")

# Invoke the chain and stream the results
for chunk in rag_chain.stream(test_query):
    print(chunk, end="", flush=True)

print("--- Execution Complete ---")

--- Running RAG chain for query: '부실시공에 따라 재시공을 하도록 한 감사건도 있나?' ---
네, 부실시공에 따라 재시공 또는 보강 조치를 하도록 한 감사 사례들이 있습니다.

다음은 관련 감사 사례들입니다.

*   **2023년도 부산경남본부 종합감사 결과 (교량내진보강공사 시공 관리 부적정)**
    *   **문제 요약:** 철근 배근이 설계도면보다 부족하게 시공되거나, 유압잭 지압응력 검토 없이 시공되는 등 부실 시공이 확인되었습니다.
    *   **조치 요약:** "설계 도면과 다르게 시공한 부분에 대해 **보강 조치 실시**" 및 "손상 및 균열이 발생한 교량 부재에 대한 안전성 검토 및 **보수 조치**"를 요구했습니다.

*   **국가철도공단 정기감사 (경부선 교량개량공사 설계 및 시공 부적정)**
    *   **문제 요약:** 설계사가 거더 솟음량을 부족하게 설계하고, 시공사가 이를 확인하고도 적절한 조치를 하지 않아 솟음량 부족으로 **보강공사가 요구**되었습니다.
    *   **조치 요약:** "적정한 대책을 마련하여 **보강공사를 시행**해야 함"을 요구했습니다.

*   **감사위원회 감사보고서(b) (발코니 출입문 설계 변경 검토 소홀)**
    *   **문제 요약:** 수급인이 설계 도면과 다르게 시공하여 발코니 출입문 유효 폭이 좁아지는 시공 상 하자가 발생했음에도, 공사비가 설계 변경으로 잘못 지급되어 공사가 부담하지 않아도 될 **재시공 공사비**가 발생했습니다. (이 사례는 이미 재시공이 이루어졌고 그 비용 회수에 대한 감사입니다.)
    *   **조치 요약:** "설계 변경으로 수급인에게 지급한 공사비에 대해 **회수 방안을 마련**"하도록 통보했습니다.

*   **2024년도 특정감사결과_14 (연약 지반 통로 박스 신축 이음 부분 이격에 따른 시공 관리 부적정)**
    *   **문제 요약:** 연약 지반 내 통로박스의 신축이음부에서 이격 현상이 발생하여 시공 및 